In [ ]:
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
import csv

In [ ]:
f = open("data/low_eng.csv",'r',encoding='utf-8-sig')
reader = csv.reader(f)
inputList = []
for line in reader:
    inputList.append(line)
f.close()

ipdf = pd.DataFrame(inputList[1:], columns=inputList[0])

DATA = []
for i in range(len(ipdf['원문'])):
    DATA.append([ipdf['원문'][i],ipdf['번역문'][i]])



In [84]:
DATA

[['\ufeff그 여자 외모는 어때요?', 'What does she look like? <END>'],
 ['그녀는 곱슬머리이다.', 'She has curly hair. <END>'],
 ['그녀는 갈색 눈을 가지고 있다.', 'She has brown eyes. <END>'],
 ['그녀는 키가 크고 예뻐요.', 'She is tall and pretty. <END>'],
 ['그녀는 성격이 어때요?', 'What is she like? <END>'],
 ['그녀는 매우 재미있어요.', 'She is very funny. <END>'],
 ['그녀는 매우 활동적이에요.', 'She is very active. <END>'],
 ['그녀는 매우 똑똑해요.', 'She is very smart. <END>'],
 ['당신은 일요일에 무엇을 하나요?', 'What do you do on Sundays? <END>'],
 ['나는 보통 도서관에 가.', 'I usually go to the library. <END>'],
 ['나는 보통 자전거를 타.', 'I usually ride a bike. <END>'],
 ['나는 보통 책을 읽어.', 'I usually read books. <END>'],
 ['나는 보통 교회에 가.', 'I usually go to church. <END>'],
 ['무슨 일이죠?', "What's the matter? <END>"],
 ['머리가 아파요.', 'I have a headache. <END>'],
 ['목이 아파요.', 'I have a sore throat. <END>'],
 ['감기에 걸렸어요.', 'I have a cold. <END>'],
 ['열이 있어요.', 'I have a fever. <END>'],
 ['너의 생각은 어때?', 'What do you think? <END>'],
 ['수영은 재미있어.', 'Swimming is fun. <END>'],
 ['조깅은 재미있어.', 'Jogging is 

In [85]:
kor_model = Word2Vec.load('model/kor.model')
eng_model = Word2Vec.load('model/eng.model')
kor_wv = kor_model.wv
eng_wv = eng_model.wv
kor_list = kor_wv.index_to_key
eng_list = eng_wv.index_to_key

In [86]:
tk_kor = []
tk_eng = []

for t in DATA:
    tk_kor.append(t[0].split(" "))
    tk_eng.append(t[1].split(" "))

In [88]:
sum_kor = []
for k in tk_kor:
    tmp = np.zeros(200)
    for v in k:
        tmp += kor_model.wv[v]
    sum_kor.append(tmp)

In [89]:
sum_eng = []
for e in tk_eng:
    tmp = np.zeros(200)
    sum_eng.append(tmp.copy())
    for v in e:
        tmp += eng_model.wv[v]
        sum_eng.append(tmp.copy())

In [90]:
X = []
X.append(np.concatenate((sum_kor[0],sum_eng[0]),axis=0))
i = 0

for j in range(1,len(sum_eng)):
    print(j)
    if (sum_eng[j] == np.zeros(200)).all():
        i += 1
    print(i)
    X.append(np.concatenate((sum_kor[i],sum_eng[j]),axis=0))


1
0
2
0
3
0
4
0
5
0
6
0
7
1
8
1
9
1
10
1
11
1
12
1
13
2
14
2
15
2
16
2
17
2
18
2
19
3
20
3
21
3
22
3
23
3
24
3
25
3
26
4
27
4
28
4
29
4
30
4
31
4
32
5
33
5
34
5
35
5
36
5
37
5
38
6
39
6
40
6
41
6
42
6
43
6
44
7
45
7
46
7
47
7
48
7
49
7
50
8
51
8
52
8
53
8
54
8
55
8
56
8
57
8
58
9
59
9
60
9
61
9
62
9
63
9
64
9
65
9
66
10
67
10
68
10
69
10
70
10
71
10
72
10
73
11
74
11
75
11
76
11
77
11
78
11
79
12
80
12
81
12
82
12
83
12
84
12
85
12
86
13
87
13
88
13
89
13
90
13
91
14
92
14
93
14
94
14
95
14
96
14
97
15
98
15
99
15
100
15
101
15
102
15
103
15
104
16
105
16
106
16
107
16
108
16
109
16
110
17
111
17
112
17
113
17
114
17
115
17
116
18
117
18
118
18
119
18
120
18
121
18
122
19
123
19
124
19
125
19
126
19
127
20
128
20
129
20
130
20
131
20
132
21
133
21
134
21
135
21
136
21
137
21
138
22
139
22
140
22
141
22
142
22
143
22
144
23
145
23
146
23
147
23
148
23
149
23
150
23
151
23
152
24
153
24
154
24
155
24
156
24
157
24
158
24
159
24
160
24
161
25
162
25
163
25
164
25
165
26
166
26
167
26
168


In [91]:
Y = []
for e in tk_eng:
    for i in range(len(e) + 1) :
        try:
            Y.append(eng_model.wv.key_to_index[e[i]])
        except:
            Y.append(len(eng_list))

In [92]:
print("Y : {}".format(len(Y)))
print("X : {}".format(len(X)))

Y : 806
X : 806


In [93]:
tov = []
for i in Y:
    tmp = np.zeros(len(eng_list) + 1)
    tmp[i] = 1
    tov.append(tmp.copy())

In [94]:
x = np.array(X)
y = np.array(tov)

In [95]:
x = x.astype(np.float32)
y = y.astype(np.float32)

In [96]:
data = {"X": x,
        "Y": y}

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


In [ ]:
class Linear_QNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = nn.Linear(input_size, hidden_size)
        self.linear2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = self.linear2(x)
        return x

    def save(self, file_name='model.pth'):
        model_folder_path = './model'
        if not os.path.exists(model_folder_path):
            os.makedirs(model_folder_path)

        file_name = os.path.join(model_folder_path, file_name)
        torch.save(self.state_dict(), file_name)

In [ ]:
model=Linear_QNet(200,128,len(eng_list) + 1)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

In [ ]:
from torchvision.transforms import ToTensor

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, Data, transform=None, target_transform=None):
        self.Y = Data["Y"]
        self.X = Data["X"]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        X = self.X[idx]
        Y = self.Y[idx]
        if self.transform:
            X = self.transform(X)
        if self.target_transform:
            Y = self.target_transform(Y)
        return X,Y

In [97]:
training_data = CustomDataset(data)

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=32, shuffle=True)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, criterion, optimizer)
print("Done!")

In [ ]:
torch.save(model, "model/low_model.pth")

In [ ]:
model.eval()

In [ ]:
qnet = torch.load("./model/test_model.pth")

In [ ]:
qnet.eval()

In [101]:
nn = torch.load("model/model.pth223")

In [102]:
nn.eval()

Linear_QNet(
  (linear1): Linear(in_features=400, out_features=256, bias=True)
  (linear2): Linear(in_features=256, out_features=246, bias=True)
  (softmax): Softmax(dim=None)
)

In [103]:
state = torch.tensor(training_data[21][0], dtype=torch.float)
predict = nn(state)
move = torch.argmax(predict)

C:\Users\진호창\Desktop\nlp 프로젝트\project\NLP_PROJECT\venv\lib\site-packages\torch\nn\modules\module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


In [104]:
print(predict)
print(move)
print(predict[move])

tensor([0.0014, 0.0038, 0.0043, 0.0040, 0.0042, 0.0041, 0.0043, 0.0043, 0.0042,
        0.0041, 0.0043, 0.0043, 0.0042, 0.0031, 0.0042, 0.0040, 0.0041, 0.0042,
        0.0043, 0.0043, 0.0043, 0.0043, 0.0042, 0.0040, 0.0043, 0.0043, 0.0043,
        0.0040, 0.0044, 0.0041, 0.0043, 0.0040, 0.0042, 0.0043, 0.0044, 0.0038,
        0.0037, 0.0041, 0.0041, 0.0043, 0.0042, 0.0030, 0.0043, 0.0044, 0.0041,
        0.0033, 0.0040, 0.0033, 0.0041, 0.0041, 0.0043, 0.0043, 0.0036, 0.0043,
        0.0043, 0.0041, 0.0042, 0.0044, 0.0038, 0.0043, 0.0028, 0.0039, 0.0043,
        0.0044, 0.0043, 0.0037, 0.0043, 0.0042, 0.0043, 0.0042, 0.0035, 0.0041,
        0.0036, 0.0043, 0.0041, 0.0044, 0.0024, 0.0042, 0.0043, 0.0041, 0.0042,
        0.0042, 0.0043, 0.0039, 0.0041, 0.0043, 0.0043, 0.0043, 0.0043, 0.0041,
        0.0036, 0.0043, 0.0043, 0.0044, 0.0033, 0.0042, 0.0040, 0.0042, 0.0042,
        0.0043, 0.0037, 0.0042, 0.0043, 0.0042, 0.0043, 0.0043, 0.0040, 0.0043,
        0.0044, 0.0044, 0.0043, 0.0043, 

In [ ]:
train_dataloader

In [ ]:
tx, ty = next(iter(train_dataloader))

In [ ]:
training_data[0]

In [ ]:
state = torch.tensor(training_data[0][0], dtype=torch.float)
predict = qnet(state)
move = torch.argmax(predict)

In [ ]:
predict[-1] = 123123123123123123

move = torch.argmax(predict)

In [ ]:
len(predict)

In [ ]:
move

In [ ]:
inputList['원문']

In [ ]:
inputList[0]

In [ ]:
ipdf

In [ ]:
ipdf['원문']

In [ ]:
DATA = []
for i in range(len(ipdf['원문'])):
    DATA.append([ipdf['원문'][i],ipdf['번역문'][i]])

In [ ]:
DATA